In [23]:
import pandas as pd

data_df = pd.read_csv("data/atec_nlp_sim_train_all.csv", sep="\t", header=None, 
                      encoding="utf-8-sig", names=["sent1", "sent2", "label"])
data_df.head(10)

,sent1,sent2,label
1,﻿怎么更改花呗手机号码,我的花呗是以前的手机号码，怎么更改成现在的支付宝的号码手机号,1
2,也开不了花呗，就这样了？完事了,真的嘛？就是花呗付款,0
3,花呗冻结以后还能开通吗,我的条件可以开通花呗借款吗,0
4,如何得知关闭借呗,想永久关闭借呗,0
5,花呗扫码付钱,二维码扫描可以用花呗吗,0
6,花呗逾期后不能分期吗,我这个 逾期后还完了 最低还款 后 能分期吗,0
7,花呗分期清空,花呗分期查询,0
8,借呗逾期短信通知,如何购买花呗短信通知,0
9,借呗即将到期要还的账单还能分期吗,借呗要分期还，是吗,0
10,花呗为什么不能支付手机交易,花呗透支了为什么不可以继续用了,0


In [24]:
import sklearn

In [25]:
from sklearn.model_selection import train_test_split

In [26]:
sent1=data_df.sent1.values[:3501]
sent2=data_df.sent2.values[:3501]
label=data_df.label.values[:3501]

In [27]:
sent1_=sent1[:2501]
sent2_=sent2[:2501]
label_=label[:2501]

In [28]:
_sent1=sent1[2501:]
_sent2=sent2[2501:]
_label=label[2501:]

In [29]:
len(_label)

1000

In [30]:
all_data=pd.DataFrame()
all_data['id_left']=range(2501)
all_data['text_left']=sent1_
all_data['id_right']=range(2501)
all_data['text_right']=sent2_
all_data['label']=label_

In [31]:
len(range(1000))

1000

In [32]:
test_data=pd.DataFrame()
test_data['id_left']=range(1000)
test_data['text_left']=_sent1
test_data['id_right']=range(1000)
test_data['text_right']=_sent2
# test_data['label']=_label

In [33]:
import matchzoo as mz

In [34]:
def load_data(df_data):
# 	df_data = pd.read_csv(data_path, sep='\t', header=None)
# 	df_data = pd.DataFrame(df_data.values, columns=['id_left', 'text_left', 'id_right', 'text_right', 'label'])
	df_data = mz.pack(df_data)
	return df_data

train_data = load_data(all_data)
# test_data1=load_data(test_data)

In [71]:
#!/usr/bin/python3
# -*- coding:utf-8 -*-
# Author:ChenYuan

import matchzoo as mz
import pandas as pd
import re  # 这个可以不用
# from sklearn import preprocessing  # 用于正则化
import numpy as np 
from matchzoo.preprocessors import BasicPreprocessor
# 这里有个Basic处理器有问题
# preprocessor = BasicPreprocessor(fixed_length_left:int=30, fixed_length_right:int=30)
preprocessor=BasicPreprocessor(15,15)
# preprocessor=mz.preprocessors.CDSSMPreprocessor()
                                                  # 定义一个数据处理器
#     ，有四种处理器，Basic是通用的、基础的数据处理器，可看官方文档，这里不做解说
                                        


In [72]:
train_dev_split = int(len(train_data) * 0.8)  # 验证集占训练数据的0.1
train = train_data[:train_dev_split]
dev = train_data[train_dev_split:]
train_pack_processed = preprocessor.fit_transform(train) 
# 其实就是做了一个字符转id操作，所以对于中文文本，不需要分词
dev_pack_processed = preprocessor.transform(dev)  
# test_pack_processed = preprocessor.transform(test_data)
train_data_generator = mz.DataGenerator(train_pack_processed
                                        , batch_size=32
                                        , shuffle=True)  # 训练数据生成器

# test_x, test_y = test_pack_processed.unpack()
dev_x, dev_y = dev_pack_processed.unpack()

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|█| 2000/2000 [00:00<00:00, 5639
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|█| 2000/2000 [00:00<00:00, 639
Building Vocabulary from a datapack.: 100%|████████████████████████████████████| 2132/2132 [00:00<00:00, 711679.76it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|█| 2000/2000 [00:00<00:00, 5767
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|█| 2000/2000 [00:00<00:00, 662
Processing text_right with transform: 100%|█████████████████████████████████████| 2000/2000 [00:00<00:00, 76973.12it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|█| 501/501 [00:00<00:00, 5063.4
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|█| 501/501 [00:00<00:00, 5509.
Processing text_right with transform: 10

# DenseBaseline

In [19]:
import matchzoo as mz
print(mz.__version__)


### 定义任务，包含两种，一个是Ranking，一个是classification
task = mz.tasks.Ranking()
print(task)

### 准备数据，数据在源码中有，不确定在pip安装的是否存在
### train_raw是matchzoo中自定的数据格式	matchzoo.data_pack.data_pack.DataPack
# train_raw = mz.datasets.toy.load_data(stage='train', task=task)
# test_raw = mz.datasets.toy.load_data(stage='test', task=task)

### 数据预处理，BasicPreprocessor为指定预处理的方式，在预处理中包含了两步
# ：fit,transform
### fit将收集一些有用的信息到preprocessor.context中
# ，不会对输入DataPack进行处理
### transformer 不会改变context、DataPack,他将重新生成转变后的DataPack.
### 在transformer过程中，包含了Tokenize => Lowercase => PuncRemoval等过程
# ，这个过程在方法中应该是可以自定义的

# preprocessor = mz.preprocessors.BasicPreprocessor()
# preprocessor.fit(train_raw)  ## init preprocessor inner state.
# train_processed = preprocessor.transform(train_raw)
# test_processed = preprocessor.transform(test_raw)

### 创建模型以及修改参数（可以使用mz.models.list_available()查看可用的模型列表）
model = mz.models.DenseBaseline()
model.params['task'] = task
model.params['mlp_num_units'] = 3
model.params.update(preprocessor.context)
model.params.completed()
model.build()
model.compile()
model.backend.summary()

### 训练, 评估, 预测
x, y = train_pack_processed .unpack()
# test_x, test_y = test_processed.unpack()
# model.fit(x , y,batch_size=32, epochs=5)
model.fit(x , y,batch_size=32, epochs=5)
print(model.evaluate(dev_x,dev_y))
# model.predict(test_x)

### 保存模型
model.save('./outputs/model/DenseBaseline-model.h5')
print('保存成功')
# loaded_model = mz.load_model('./outputs/model/DenseBaseline-model.h5')

2.2.0
Ranking Task
Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 15)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 15)           0                                            
__________________________________________________________________________________________________
concatenate_6 (Concatenate)     (None, 30)           0           text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
dense_26 (Dense)                (None, 3)            93          concaten

# ArcI 

In [39]:
import matchzoo as mz
print(mz.__version__)


### 定义任务，包含两种，一个是Ranking，一个是classification
task = mz.tasks.Ranking()
print(task)

### 准备数据，数据在源码中有，不确定在pip安装的是否存在
### train_raw是matchzoo中自定的数据格式	matchzoo.data_pack.data_pack.DataPack
# train_raw = mz.datasets.toy.load_data(stage='train', task=task)
# test_raw = mz.datasets.toy.load_data(stage='test', task=task)

### 数据预处理，BasicPreprocessor为指定预处理的方式，在预处理中包含了两步
# ：fit,transform
### fit将收集一些有用的信息到preprocessor.context中
# ，不会对输入DataPack进行处理
### transformer 不会改变context、DataPack,他将重新生成转变后的DataPack.
### 在transformer过程中，包含了Tokenize => Lowercase => PuncRemoval等过程
# ，这个过程在方法中应该是可以自定义的

# preprocessor = mz.preprocessors.BasicPreprocessor()
# preprocessor.fit(train_raw)  ## init preprocessor inner state.
# train_processed = preprocessor.transform(train_raw)
# test_processed = preprocessor.transform(test_raw)

### 创建模型以及修改参数（可以使用mz.models.list_available()查看可用的模型列表）
# model = mz.models.DenseBaseline()
# model.params['task'] = task
# model.params['mlp_num_units'] = 3
# model.params.update(preprocessor.context)
# model.params.completed()
# model.build()
# model.compile()
# model.backend.summary()

# >>> model = ANMM()
# model.guess_and_fill_missing_params(verbose=0)
# model.build()

        
model =mz.models.ArcI()
model.params['num_blocks'] = 1
model.params['left_filters'] = [32]
model.params['right_filters'] = [32]
model.params['left_kernel_sizes'] = [3]
model.params['right_kernel_sizes'] = [3]
model.params['left_pool_sizes'] = [2]
model.params['right_pool_sizes'] = [4]
model.params['conv_activation_func'] = 'relu'
model.params['mlp_num_layers'] = 1
model.params['mlp_num_units'] = 64
model.params['mlp_num_fan_out'] = 32
model.params['mlp_activation_func'] = 'relu'
model.params['dropout_rate'] = 0.5
model.guess_and_fill_missing_params(verbose=0)
model.params.update(preprocessor.context)
model.params.completed()
model.build()
model.compile()
model.backend.summary()

### 训练, 评估, 预测
x, y = train_pack_processed .unpack()
# test_x, test_y = test_processed.unpack()
# model.fit(x , y,batch_size=32, epochs=5)
model.fit(x , y,batch_size=32, epochs=5)
print(model.evaluate(dev_x,dev_y))
# model.predict(test_x)

### 保存模型
model.save('./outputs/model/ANMM.h5')
print('保存成功')
# loaded_model = mz.load_model('./outputs/model/DenseBaseline-model.h5')

2.2.0
Ranking Task
Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 15)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 15)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 15, 300)      629700      text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 15, 32)       28832       embeddin

C:\Users\Administrator\Anaconda3\lib\site-packages\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
2000/2000 [==============================] - 3s 2ms/step - loss: 0.1616
Epoch 2/5
2000/2000 [==============================] - 3s 1ms/step - loss: 0.1578
Epoch 3/5
2000/2000 [==============================] - 3s 1ms/step - loss: 0.1450
Epoch 4/5
2000/2000 [==============================] - 3s 1ms/step - loss: 0.0372
Epoch 5/5
2000/2000 [==============================] - 3s 1ms/step - loss: 0.0241
{mean_average_precision(0.0): 0.18562874251497005}
保存成功


# ConvKNRM

In [43]:
import matchzoo as mz
print(mz.__version__)


### 定义任务，包含两种，一个是Ranking，一个是classification
task = mz.tasks.Ranking()
print(task)

### 准备数据，数据在源码中有，不确定在pip安装的是否存在
### train_raw是matchzoo中自定的数据格式	matchzoo.data_pack.data_pack.DataPack
# train_raw = mz.datasets.toy.load_data(stage='train', task=task)
# test_raw = mz.datasets.toy.load_data(stage='test', task=task)

### 数据预处理，BasicPreprocessor为指定预处理的方式，在预处理中包含了两步
# ：fit,transform
### fit将收集一些有用的信息到preprocessor.context中
# ，不会对输入DataPack进行处理
### transformer 不会改变context、DataPack,他将重新生成转变后的DataPack.
### 在transformer过程中，包含了Tokenize => Lowercase => PuncRemoval等过程
# ，这个过程在方法中应该是可以自定义的

# preprocessor = mz.preprocessors.BasicPreprocessor()
# preprocessor.fit(train_raw)  ## init preprocessor inner state.
# train_processed = preprocessor.transform(train_raw)
# test_processed = preprocessor.transform(test_raw)

### 创建模型以及修改参数（可以使用mz.models.list_available()查看可用的模型列表）
# model = mz.models.DenseBaseline()
# model.params['task'] = task
# model.params['mlp_num_units'] = 3
# model.params.update(preprocessor.context)
# model.params.completed()
# model.build()
# model.compile()
# model.backend.summary()

# >>> model = ANMM()
# model.guess_and_fill_missing_params(verbose=0)
# model.build()


model = mz.models.ConvKNRM()
model.params['embedding_input_dim'] = 10000
model.params['embedding_output_dim'] = 300
model.params['embedding_trainable'] = True
model.params['filters'] = 128
model.params['conv_activation_func'] = 'tanh'
model.params['max_ngram'] = 3
model.params['use_crossmatch'] = True
model.params['kernel_num'] = 11
model.params['sigma'] = 0.1
model.params['exact_sigma'] = 0.001
model.guess_and_fill_missing_params(verbose=0)
model.params.update(preprocessor.context)
model.params.completed()
model.build()
model.compile()
model.backend.summary()

# model =mz.models.DRMM()
# model.params['mlp_num_layers'] = 1
# model.params['mlp_num_units'] = 5
# model.params['mlp_num_fan_out'] = 1
# model.params['mlp_activation_func'] = 'tanh'
# model.guess_and_fill_missing_params(verbose=0)
# model.params.update(preprocessor.context)
# model.params.completed()
# model.build()
# model.compile()
# model.backend.summary()

        
# model =mz.models.ArcI()
# model.params['num_blocks'] = 1
# model.params['left_filters'] = [32]
# model.params['right_filters'] = [32]
# model.params['left_kernel_sizes'] = [3]
# model.params['right_kernel_sizes'] = [3]
# model.params['left_pool_sizes'] = [2]
# model.params['right_pool_sizes'] = [4]
# model.params['conv_activation_func'] = 'relu'
# model.params['mlp_num_layers'] = 1
# model.params['mlp_num_units'] = 64
# model.params['mlp_num_fan_out'] = 32
# model.params['mlp_activation_func'] = 'relu'
# model.params['dropout_rate'] = 0.5
# model.guess_and_fill_missing_params(verbose=0)
# model.params.update(preprocessor.context)
# model.params.completed()
# model.build()
# model.compile()
# model.backend.summary()

### 训练, 评估, 预测
x, y = train_pack_processed .unpack()
# test_x, test_y = test_processed.unpack()
# model.fit(x , y,batch_size=32, epochs=5)
model.fit(x , y,batch_size=32, epochs=5)
print(model.evaluate(dev_x,dev_y))
# model.predict(test_x)

### 保存模型
model.save('./outputs/model/ConvKNRM.h5')
print('保存成功')
# loaded_model = mz.load_model('./outputs/model/DenseBaseline-model.h5')

2.2.0
Ranking Task
Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 15)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 15)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 15, 300)      629700      text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 15, 128)      38528       embeddin

C:\Users\Administrator\Anaconda3\lib\site-packages\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


lambda_64 (Lambda)              (None, 15)           0           activation_64[0][0]              
__________________________________________________________________________________________________
lambda_66 (Lambda)              (None, 15)           0           activation_66[0][0]              
__________________________________________________________________________________________________
lambda_68 (Lambda)              (None, 15)           0           activation_68[0][0]              
__________________________________________________________________________________________________
lambda_70 (Lambda)              (None, 15)           0           activation_70[0][0]              
__________________________________________________________________________________________________
lambda_72 (Lambda)              (None, 15)           0           activation_72[0][0]              
__________________________________________________________________________________________________
lambda_74 

2000/2000 [==============================] - 12s 6ms/step - loss: 4.2023
Epoch 2/5
2000/2000 [==============================] - 5s 3ms/step - loss: 0.5019
Epoch 3/5
2000/2000 [==============================] - 6s 3ms/step - loss: 0.1633
Epoch 4/5
2000/2000 [==============================] - 6s 3ms/step - loss: 0.1223
Epoch 5/5
2000/2000 [==============================] - 6s 3ms/step - loss: 0.0795
{mean_average_precision(0.0): 0.18562874251497005}
保存成功


In [45]:
mz.models.list_available()

[matchzoo.models.naive.Naive,
 matchzoo.models.dssm.DSSM,
 matchzoo.models.cdssm.CDSSM,
 matchzoo.models.dense_baseline.DenseBaseline,
 matchzoo.models.arci.ArcI,
 matchzoo.models.arcii.ArcII,
 matchzoo.models.match_pyramid.MatchPyramid,
 matchzoo.models.knrm.KNRM,
 matchzoo.models.duet.DUET,
 matchzoo.models.drmmtks.DRMMTKS,
 matchzoo.models.drmm.DRMM,
 matchzoo.models.anmm.ANMM,
 matchzoo.models.mvlstm.MVLSTM,
 matchzoo.contrib.models.match_lstm.MatchLSTM,
 matchzoo.contrib.models.match_srnn.MatchSRNN,
 matchzoo.contrib.models.hbmp.HBMP,
 matchzoo.contrib.models.esim.ESIM,
 matchzoo.contrib.models.bimpm.BiMPM,
 matchzoo.contrib.models.diin.DIIN,
 matchzoo.models.conv_knrm.ConvKNRM]

# DRMMTKS

In [48]:
import matchzoo as mz
print(mz.__version__)


### 定义任务，包含两种，一个是Ranking，一个是classification
task = mz.tasks.Ranking()
print(task)



# model = mz.models.ConvKNRM()
# model.params['embedding_input_dim'] = 10000
# model.params['embedding_output_dim'] = 300
# model.params['embedding_trainable'] = True
# model.params['filters'] = 128
# model.params['conv_activation_func'] = 'tanh'
# model.params['max_ngram'] = 3
# model.params['use_crossmatch'] = True
# model.params['kernel_num'] = 11
# model.params['sigma'] = 0.1
# model.params['exact_sigma'] = 0.001
# model.guess_and_fill_missing_params(verbose=0)
# model.params.update(preprocessor.context)
# model.params.completed()
# model.build()
# model.compile()
# model.backend.summary()

model =mz.models.DRMMTKS()
model.params['embedding_input_dim'] = 10000
model.params['embedding_output_dim'] = 100
model.params['top_k'] = 20
model.params['mlp_num_layers'] = 1
model.params['mlp_num_units'] = 5
model.params['mlp_num_fan_out'] = 1
model.params['mlp_activation_func'] = 'tanh'
model.guess_and_fill_missing_params(verbose=0)
# model.build()
model.params.update(preprocessor.context)
model.params.completed()
model.build()
model.compile()
model.backend.summary()

        
# model =mz.models.ArcI()
# model.params['num_blocks'] = 1
# model.params['left_filters'] = [32]
# model.params['right_filters'] = [32]
# model.params['left_kernel_sizes'] = [3]
# model.params['right_kernel_sizes'] = [3]
# model.params['left_pool_sizes'] = [2]
# model.params['right_pool_sizes'] = [4]
# model.params['conv_activation_func'] = 'relu'
# model.params['mlp_num_layers'] = 1
# model.params['mlp_num_units'] = 64
# model.params['mlp_num_fan_out'] = 32
# model.params['mlp_activation_func'] = 'relu'
# model.params['dropout_rate'] = 0.5
# model.guess_and_fill_missing_params(verbose=0)
# model.params.update(preprocessor.context)
# model.params.completed()
# model.build()
# model.compile()
# model.backend.summary()

### 训练, 评估, 预测
x, y = train_pack_processed .unpack()
# test_x, test_y = test_processed.unpack()
# model.fit(x , y,batch_size=32, epochs=5)
model.fit(x , y,batch_size=32, epochs=5)
print(model.evaluate(dev_x,dev_y))
# model.predict(test_x)

### 保存模型
model.save('./outputs/model/DRMMTKS.h5')
print('保存成功')
# loaded_model = mz.load_model('./outputs/model/DenseBaseline-model.h5')

2.2.0
Ranking Task
Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 15)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 15)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 15, 100)      209900      text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
dot_27 (Dot)                    (None, 15, 15)       0           embeddi

C:\Users\Administrator\Anaconda3\lib\site-packages\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
2000/2000 [==============================] - 1s 523us/step - loss: 0.1690
Epoch 2/5
2000/2000 [==============================] - 1s 319us/step - loss: 0.1534
Epoch 3/5
2000/2000 [==============================] - 1s 323us/step - loss: 0.1217
Epoch 4/5
2000/2000 [==============================] - 1s 358us/step - loss: 0.0538
Epoch 5/5
2000/2000 [==============================] - 1s 297us/step - loss: 0.0271
{mean_average_precision(0.0): 0.18562874251497005}
保存成功


# DUET

In [49]:
import matchzoo as mz
print(mz.__version__)


### 定义任务，包含两种，一个是Ranking，一个是classification
task = mz.tasks.Ranking()
print(task)



# model = mz.models.ConvKNRM()
model = mz.models.DUET()
model.params['embedding_input_dim'] = 1000
model.params['embedding_output_dim'] = 300
model.params['lm_filters'] = 32
model.params['lm_hidden_sizes'] = [64, 32]
model.params['dropout_rate'] = 0.5
model.params['dm_filters'] = 32
model.params['dm_kernel_size'] = 3
model.params['dm_d_mpool'] = 4
model.params['dm_hidden_sizes'] = [64, 32]
model.guess_and_fill_missing_params(verbose=0)
model.params.update(preprocessor.context)
model.params.completed()
model.build()
model.compile()
model.backend.summary()

        
# model =mz.models.ArcI()
# model.params['num_blocks'] = 1
# model.params['left_filters'] = [32]
# model.params['right_filters'] = [32]
# model.params['left_kernel_sizes'] = [3]
# model.params['right_kernel_sizes'] = [3]
# model.params['left_pool_sizes'] = [2]
# model.params['right_pool_sizes'] = [4]
# model.params['conv_activation_func'] = 'relu'
# model.params['mlp_num_layers'] = 1
# model.params['mlp_num_units'] = 64
# model.params['mlp_num_fan_out'] = 32
# model.params['mlp_activation_func'] = 'relu'
# model.params['dropout_rate'] = 0.5
# model.guess_and_fill_missing_params(verbose=0)
# model.params.update(preprocessor.context)
# model.params.completed()
# model.build()
# model.compile()
# model.backend.summary()

### 训练, 评估, 预测
x, y = train_pack_processed .unpack()
# test_x, test_y = test_processed.unpack()
# model.fit(x , y,batch_size=32, epochs=5)
model.fit(x , y,batch_size=32, epochs=5)
print(model.evaluate(dev_x,dev_y))
# model.predict(test_x)

### 保存模型
model.save('./outputs/model/DUET.h5')
print('保存成功')
# loaded_model = mz.load_model('./outputs/model/DenseBaseline-model.h5')

2.2.0
Ranking Task
Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 15)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 15)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 15, 300)      629700      text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
conv1d_12 (Conv1D)              (None, 15, 32)       28832       embeddi

C:\Users\Administrator\Anaconda3\lib\site-packages\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
2000/2000 [==============================] - 5s 2ms/step - loss: 0.2327
Epoch 2/5
2000/2000 [==============================] - 3s 2ms/step - loss: 0.1600
Epoch 3/5
2000/2000 [==============================] - 3s 1ms/step - loss: 0.1573
Epoch 4/5
2000/2000 [==============================] - 3s 2ms/step - loss: 0.1337
Epoch 5/5
2000/2000 [==============================] - 3s 1ms/step - loss: 0.0733
{mean_average_precision(0.0): 0.18562874251497005}
保存成功


# ESIM

In [56]:
import matchzoo as mz
print(mz.__version__)
import tensorflow.keras as K

### 定义任务，包含两种，一个是Ranking，一个是classification
task = mz.tasks.Ranking()
print(task)



# model = mz.models.ConvKNRM()
model = mz.contrib.models.ESIM()
# task = classification_task = mz.tasks.Classification(num_classes=2)
model.params['task'] = task
model.params['input_shapes'] = [(15, ), (15, )]
model.params['lstm_dim'] = 300
model.params['mlp_num_units'] = 300
model.params['embedding_input_dim'] =  5000
model.params['embedding_output_dim'] =  10
model.params['embedding_trainable'] = False
model.params['mlp_num_layers'] = 0
model.params['mlp_num_fan_out'] = 300
model.params['mlp_activation_func'] = 'tanh'
model.params['mask_value'] = 0
model.params['dropout_rate'] = 0.5
model.params['optimizer'] = K.optimizers.Adam(lr=4e-4)
model.guess_and_fill_missing_params()
# model.build()
model.params.update(preprocessor.context)
model.params.completed()
model.build()
model.compile()
model.backend.summary()

        
# model =mz.models.ArcI()
# model.params['num_blocks'] = 1
# model.params['left_filters'] = [32]
# model.params['right_filters'] = [32]
# model.params['left_kernel_sizes'] = [3]
# model.params['right_kernel_sizes'] = [3]
# model.params['left_pool_sizes'] = [2]
# model.params['right_pool_sizes'] = [4]
# model.params['conv_activation_func'] = 'relu'
# model.params['mlp_num_layers'] = 1
# model.params['mlp_num_units'] = 64
# model.params['mlp_num_fan_out'] = 32
# model.params['mlp_activation_func'] = 'relu'
# model.params['dropout_rate'] = 0.5
# model.guess_and_fill_missing_params(verbose=0)
# model.params.update(preprocessor.context)
# model.params.completed()
# model.build()
# model.compile()
# model.backend.summary()

### 训练, 评估, 预测
x, y = train_pack_processed .unpack()
# test_x, test_y = test_processed.unpack()
# model.fit(x , y,batch_size=32, epochs=5)
model.fit(x , y,batch_size=32, epochs=5)
print(model.evaluate(dev_x,dev_y))
# model.predict(test_x)

### 保存模型
model.save('./outputs/model/ESIM.h5')
print('保存成功')
# loaded_model = mz.load_model('./outputs/model/DenseBaseline-model.h5')

2.2.0
Ranking Task
Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 15)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 15)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 15, 10)       20990       text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
dropout_12 (Dropout)            multiple             0           embeddi

Epoch 1/5
2000/2000 [==============================] - 67s 34ms/step - loss: nan
Epoch 2/5
2000/2000 [==============================] - 61s 31ms/step - loss: nan
Epoch 3/5
2000/2000 [==============================] - 64s 32ms/step - loss: nan
Epoch 4/5
2000/2000 [==============================] - 70s 35ms/step - loss: nan
Epoch 5/5
2000/2000 [==============================] - 67s 33ms/step - loss: nan
{mean_average_precision(0.0): 0.18562874251497005}
保存成功


In [50]:
mz.models.list_available()

[matchzoo.models.naive.Naive,
 matchzoo.models.dssm.DSSM,
 matchzoo.models.cdssm.CDSSM,
 matchzoo.models.dense_baseline.DenseBaseline,
 matchzoo.models.arci.ArcI,
 matchzoo.models.arcii.ArcII,
 matchzoo.models.match_pyramid.MatchPyramid,
 matchzoo.models.knrm.KNRM,
 matchzoo.models.duet.DUET,
 matchzoo.models.drmmtks.DRMMTKS,
 matchzoo.models.drmm.DRMM,
 matchzoo.models.anmm.ANMM,
 matchzoo.models.mvlstm.MVLSTM,
 matchzoo.contrib.models.match_lstm.MatchLSTM,
 matchzoo.contrib.models.match_srnn.MatchSRNN,
 matchzoo.contrib.models.hbmp.HBMP,
 matchzoo.contrib.models.esim.ESIM,
 matchzoo.contrib.models.bimpm.BiMPM,
 matchzoo.contrib.models.diin.DIIN,
 matchzoo.models.conv_knrm.ConvKNRM]

# HBMP

In [57]:
import matchzoo as mz
print(mz.__version__)
import tensorflow.keras as K

### 定义任务，包含两种，一个是Ranking，一个是classification
task = mz.tasks.Ranking()
print(task)





model = mz.contrib.models.HBMP()
model.guess_and_fill_missing_params(verbose=0)
model.params['embedding_input_dim'] = 200
model.params['embedding_output_dim'] = 100
model.params['embedding_trainable'] = True
model.params['alpha'] = 0.1
model.params['mlp_num_layers'] = 3
model.params['mlp_num_units'] = [10, 10]
model.params['lstm_num_units'] = 5
model.params['dropout_rate'] = 0.1
# model.build()

# model.build()
model.params.update(preprocessor.context)
model.params.completed()
model.build()
model.compile()
model.backend.summary()

        
# model =mz.models.ArcI()
# model.params['num_blocks'] = 1
# model.params['left_filters'] = [32]
# model.params['right_filters'] = [32]
# model.params['left_kernel_sizes'] = [3]
# model.params['right_kernel_sizes'] = [3]
# model.params['left_pool_sizes'] = [2]
# model.params['right_pool_sizes'] = [4]
# model.params['conv_activation_func'] = 'relu'
# model.params['mlp_num_layers'] = 1
# model.params['mlp_num_units'] = 64
# model.params['mlp_num_fan_out'] = 32
# model.params['mlp_activation_func'] = 'relu'
# model.params['dropout_rate'] = 0.5
# model.guess_and_fill_missing_params(verbose=0)
# model.params.update(preprocessor.context)
# model.params.completed()
# model.build()
# model.compile()
# model.backend.summary()

### 训练, 评估, 预测
x, y = train_pack_processed .unpack()
# test_x, test_y = test_processed.unpack()
# model.fit(x , y,batch_size=32, epochs=5)
model.fit(x , y,batch_size=32, epochs=5)
print(model.evaluate(dev_x,dev_y))
# model.predict(test_x)

### 保存模型
model.save('./outputs/model/HBMP.h5')
print('保存成功')
# loaded_model = mz.load_model('./outputs/model/DenseBaseline-model.h5')

2.2.0
Ranking Task
Model: "model_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 15)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 15)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 15, 100)      209900      text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
bidirectional_3 (Bidirectional) [(None, 15, 10), (No 4240        embeddi

C:\Users\Administrator\Anaconda3\lib\site-packages\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


leaky_re_lu_1 (LeakyReLU)       (None, 10)           0           dense_47[0][0]                   
__________________________________________________________________________________________________
dropout_14 (Dropout)            (None, 10)           0           leaky_re_lu_1[0][0]              
__________________________________________________________________________________________________
dense_48 (Dense)                (None, 10)           110         dropout_14[0][0]                 
__________________________________________________________________________________________________
leaky_re_lu_2 (LeakyReLU)       (None, 10)           0           dense_48[0][0]                   
__________________________________________________________________________________________________
dense_49 (Dense)                (None, 1)            11          leaky_re_lu_2[0][0]              
Total params: 236,671
Trainable params: 236,671
Non-trainable params: 0
_____________________________________

# KNRM

In [58]:
import matchzoo as mz
print(mz.__version__)
import tensorflow.keras as K

### 定义任务，包含两种，一个是Ranking，一个是classification
task = mz.tasks.Ranking()
print(task)





# model = mz.contrib.models.HBMP()
# model.guess_and_fill_missing_params(verbose=0)
# model.params['embedding_input_dim'] = 200
# model.params['embedding_output_dim'] = 100
# model.params['embedding_trainable'] = True
# model.params['alpha'] = 0.1
# model.params['mlp_num_layers'] = 3
# model.params['mlp_num_units'] = [10, 10]
# model.params['lstm_num_units'] = 5
# model.params['dropout_rate'] = 0.1
# model.build()

model =mz.models.KNRM()
model.params['embedding_input_dim'] =  10000
model.params['embedding_output_dim'] =  10
model.params['embedding_trainable'] = True
model.params['kernel_num'] = 11
model.params['sigma'] = 0.1
model.params['exact_sigma'] = 0.001
model.guess_and_fill_missing_params(verbose=0)
# model.build()

# model.build()
model.params.update(preprocessor.context)
model.params.completed()
model.build()
model.compile()
model.backend.summary()


### 训练, 评估, 预测
x, y = train_pack_processed .unpack()
# test_x, test_y = test_processed.unpack()
# model.fit(x , y,batch_size=32, epochs=5)
model.fit(x , y,batch_size=32, epochs=5)
print(model.evaluate(dev_x,dev_y))
# model.predict(test_x)

### 保存模型
model.save('./outputs/model/KNRM.h5')
print('保存成功')
# loaded_model = mz.load_model('./outputs/model/DenseBaseline-model.h5')

2.2.0
Ranking Task
Model: "model_14"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 15)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 15)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 15, 10)       20990       text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
dot_32 (Dot)                    (None, 15, 15)       0           embeddi

C:\Users\Administrator\Anaconda3\lib\site-packages\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
2000/2000 [==============================] - 1s 582us/step - loss: 422.1236
Epoch 2/5
2000/2000 [==============================] - 0s 148us/step - loss: 242.0767
Epoch 3/5
2000/2000 [==============================] - 0s 151us/step - loss: 129.8622
Epoch 4/5
2000/2000 [==============================] - 0s 147us/step - loss: 65.8162
Epoch 5/5
2000/2000 [==============================] - 0s 145us/step - loss: 30.3271
{mean_average_precision(0.0): 0.18562874251497005}
保存成功


# MatchLSTM

In [61]:
import matchzoo as mz
print(mz.__version__)
import tensorflow.keras as K

### 定义任务，包含两种，一个是Ranking，一个是classification
task = mz.tasks.Ranking()
print(task)



# model = mz.contrib.models.HBMP()
# model.guess_and_fill_missing_params(verbose=0)
# model.params['embedding_input_dim'] = 200
# model.params['embedding_output_dim'] = 100
# model.params['embedding_trainable'] = True
# model.params['alpha'] = 0.1
# model.params['mlp_num_layers'] = 3
# model.params['mlp_num_units'] = [10, 10]
# model.params['lstm_num_units'] = 5
# model.params['dropout_rate'] = 0.1
# model.build()

model =mz.contrib.models.MatchLSTM()
# model = MatchLSTM()
model.guess_and_fill_missing_params(verbose=0)
model.params['embedding_input_dim'] = 10000
model.params['embedding_output_dim'] = 100
model.params['embedding_trainable'] = True
model.params['fc_num_units'] = 200
model.params['lstm_num_units'] = 256
model.params['dropout_rate'] = 0.5
model.params.update(preprocessor.context)
model.params.completed()
model.build()
model.compile()
model.backend.summary()


### 训练, 评估, 预测
x, y = train_pack_processed .unpack()
# test_x, test_y = test_processed.unpack()
# model.fit(x , y,batch_size=32, epochs=5)
model.fit(x , y,batch_size=32, epochs=5)
print(model.evaluate(dev_x,dev_y))
# model.predict(test_x)

### 保存模型
model.save('./outputs/model/MatchLSTM.h5')
print('保存成功')
# loaded_model = mz.load_model('./outputs/model/DenseBaseline-model.h5')

2.2.0
Ranking Task
Model: "model_15"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 15)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 15)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 15, 100)      209900      text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
lstm_left (LSTM)                (None, 15, 256)      365568      embeddi

C:\Users\Administrator\Anaconda3\lib\site-packages\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
2000/2000 [==============================] - 59s 29ms/step - loss: 0.1630
Epoch 2/5
2000/2000 [==============================] - 57s 29ms/step - loss: 0.1583
Epoch 3/5
2000/2000 [==============================] - 57s 29ms/step - loss: 0.1579
Epoch 4/5
2000/2000 [==============================] - 58s 29ms/step - loss: 0.1592
Epoch 5/5
2000/2000 [==============================] - 59s 29ms/step - loss: 0.1584
{mean_average_precision(0.0): 0.18562874251497005}
保存成功


In [59]:
mz.models.list_available()

[matchzoo.models.naive.Naive,
 matchzoo.models.dssm.DSSM,
 matchzoo.models.cdssm.CDSSM,
 matchzoo.models.dense_baseline.DenseBaseline,
 matchzoo.models.arci.ArcI,
 matchzoo.models.arcii.ArcII,
 matchzoo.models.match_pyramid.MatchPyramid,
 matchzoo.models.knrm.KNRM,
 matchzoo.models.duet.DUET,
 matchzoo.models.drmmtks.DRMMTKS,
 matchzoo.models.drmm.DRMM,
 matchzoo.models.anmm.ANMM,
 matchzoo.models.mvlstm.MVLSTM,
 matchzoo.contrib.models.match_lstm.MatchLSTM,
 matchzoo.contrib.models.match_srnn.MatchSRNN,
 matchzoo.contrib.models.hbmp.HBMP,
 matchzoo.contrib.models.esim.ESIM,
 matchzoo.contrib.models.bimpm.BiMPM,
 matchzoo.contrib.models.diin.DIIN,
 matchzoo.models.conv_knrm.ConvKNRM]

# MatchSRNN

In [62]:
import matchzoo as mz
print(mz.__version__)
import tensorflow.keras as K

### 定义任务，包含两种，一个是Ranking，一个是classification
task = mz.tasks.Ranking()
print(task)



# model = mz.contrib.models.HBMP()
# model.guess_and_fill_missing_params(verbose=0)
# model.params['embedding_input_dim'] = 200
# model.params['embedding_output_dim'] = 100
# model.params['embedding_trainable'] = True
# model.params['alpha'] = 0.1
# model.params['mlp_num_layers'] = 3
# model.params['mlp_num_units'] = [10, 10]
# model.params['lstm_num_units'] = 5
# model.params['dropout_rate'] = 0.1
# model.build()

model =mz.contrib.models.MatchSRNN()
model.params['channels'] = 4
model.params['units'] = 10
model.params['dropout_rate'] = 0.0
model.params['direction'] = 'lt'
model.guess_and_fill_missing_params(verbose=0)
model.params.update(preprocessor.context)
model.params.completed()
model.build()
model.compile()
model.backend.summary()


### 训练, 评估, 预测
x, y = train_pack_processed .unpack()
# test_x, test_y = test_processed.unpack()
# model.fit(x , y,batch_size=32, epochs=5)
model.fit(x , y,batch_size=32, epochs=5)
print(model.evaluate(dev_x,dev_y))
# model.predict(test_x)

### 保存模型
model.save('./outputs/model/MatchSRNN.h5')
print('保存成功')
# loaded_model = mz.load_model('./outputs/model/DenseBaseline-model.h5')

2.2.0
Ranking Task
Model: "model_16"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 15)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 15)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 15, 300)      629700      text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
matching_tensor_layer_1 (Matchi (None, 4, 15, 15)    360000      embeddi

C:\Users\Administrator\Anaconda3\lib\site-packages\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
2000/2000 [==============================] - 32s 16ms/step - loss: 0.1644
Epoch 2/5
2000/2000 [==============================] - 32s 16ms/step - loss: 0.1575
Epoch 3/5
2000/2000 [==============================] - 32s 16ms/step - loss: 0.1235
Epoch 4/5
2000/2000 [==============================] - 30s 15ms/step - loss: 0.0139
Epoch 5/5
2000/2000 [==============================] - 31s 15ms/step - loss: 0.0035
{mean_average_precision(0.0): 0.18562874251497005}
保存成功


In [63]:
mz.models.list_available()

[matchzoo.models.naive.Naive,
 matchzoo.models.dssm.DSSM,
 matchzoo.models.cdssm.CDSSM,
 matchzoo.models.dense_baseline.DenseBaseline,
 matchzoo.models.arci.ArcI,
 matchzoo.models.arcii.ArcII,
 matchzoo.models.match_pyramid.MatchPyramid,
 matchzoo.models.knrm.KNRM,
 matchzoo.models.duet.DUET,
 matchzoo.models.drmmtks.DRMMTKS,
 matchzoo.models.drmm.DRMM,
 matchzoo.models.anmm.ANMM,
 matchzoo.models.mvlstm.MVLSTM,
 matchzoo.contrib.models.match_lstm.MatchLSTM,
 matchzoo.contrib.models.match_srnn.MatchSRNN,
 matchzoo.contrib.models.hbmp.HBMP,
 matchzoo.contrib.models.esim.ESIM,
 matchzoo.contrib.models.bimpm.BiMPM,
 matchzoo.contrib.models.diin.DIIN,
 matchzoo.models.conv_knrm.ConvKNRM]

# MVLSTM

In [64]:
import matchzoo as mz
print(mz.__version__)
import tensorflow.keras as K

### 定义任务，包含两种，一个是Ranking，一个是classification
task = mz.tasks.Ranking()
print(task)



# model = mz.contrib.models.HBMP()
# model.guess_and_fill_missing_params(verbose=0)
# model.params['embedding_input_dim'] = 200
# model.params['embedding_output_dim'] = 100
# model.params['embedding_trainable'] = True
# model.params['alpha'] = 0.1
# model.params['mlp_num_layers'] = 3
# model.params['mlp_num_units'] = [10, 10]
# model.params['lstm_num_units'] = 5
# model.params['dropout_rate'] = 0.1
# model.build()

model =mz.models.MVLSTM()
model.params['lstm_units'] = 32
model.params['top_k'] = 50
model.params['mlp_num_layers'] = 2
model.params['mlp_num_units'] = 20
model.params['mlp_num_fan_out'] = 10
model.params['mlp_activation_func'] = 'relu'
model.params['dropout_rate'] = 0.5
model.guess_and_fill_missing_params(verbose=0)

model.params.update(preprocessor.context)
model.params.completed()
model.build()
model.compile()
model.backend.summary()


### 训练, 评估, 预测
x, y = train_pack_processed .unpack()
# test_x, test_y = test_processed.unpack()
# model.fit(x , y,batch_size=32, epochs=5)
model.fit(x , y,batch_size=32, epochs=5)
print(model.evaluate(dev_x,dev_y))
# model.predict(test_x)

### 保存模型
model.save('./outputs/model/MVLSTM.h5')
print('保存成功')
# loaded_model = mz.load_model('./outputs/model/DenseBaseline-model.h5')

2.2.0
Ranking Task
Model: "model_17"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 15)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 15)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 15, 300)      629700      text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
bidirectional_9 (Bidirectional) (None, 15, 64)       85248       embeddi

C:\Users\Administrator\Anaconda3\lib\site-packages\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
2000/2000 [==============================] - 9s 5ms/step - loss: 0.1712
Epoch 2/5
2000/2000 [==============================] - 7s 3ms/step - loss: 0.1639
Epoch 3/5
2000/2000 [==============================] - 7s 4ms/step - loss: 0.1616
Epoch 4/5
2000/2000 [==============================] - 8s 4ms/step - loss: 0.1597
Epoch 5/5
2000/2000 [==============================] - 8s 4ms/step - loss: 0.1576
{mean_average_precision(0.0): 0.18562874251497005}
保存成功


# BiMPM

In [73]:
import matchzoo as mz
print(mz.__version__)
import tensorflow.keras as K

### 定义任务，包含两种，一个是Ranking，一个是classification
task = mz.tasks.Ranking()
print(task)



# model = mz.contrib.models.HBMP()
# model.guess_and_fill_missing_params(verbose=0)
# model.params['embedding_input_dim'] = 200
# model.params['embedding_output_dim'] = 100
# model.params['embedding_trainable'] = True
# model.params['alpha'] = 0.1
# model.params['mlp_num_layers'] = 3
# model.params['mlp_num_units'] = [10, 10]
# model.params['lstm_num_units'] = 5
# model.params['dropout_rate'] = 0.1
# model.build()

model = mz.contrib.models.BiMPM()
model.guess_and_fill_missing_params(verbose=0)
# model.build()
model.params.update(preprocessor.context)
model.params.completed()
model.build()
model.compile()
model.backend.summary()


### 训练, 评估, 预测
x, y = train_pack_processed .unpack()
# test_x, test_y = test_processed.unpack()
# model.fit(x , y,batch_size=32, epochs=5)
model.fit(x , y,batch_size=32, epochs=5)
print(model.evaluate(dev_x,dev_y))
# model.predict(test_x)

### 保存模型
model.save('./outputs/model/BiMPM.h5')
print('保存成功')
# loaded_model = mz.load_model('./outputs/model/DenseBaseline-model.h5')

2.2.0
Ranking Task
Model: "model_20"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 15)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 15)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 15, 300)      629700      text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
dropout_21 (Dropout)            (None, 15, 300)      0           embeddi

C:\Users\Administrator\Anaconda3\lib\site-packages\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "



__________________________________________________________________________________________________
dropout_24 (Dropout)            (None, 15, 15)       0           multi_perspective_layer_1[1][0]  
__________________________________________________________________________________________________
bidirectional_12 (Bidirectional (None, 8)            640         dropout_23[0][0]                 
                                                                 dropout_24[0][0]                 
__________________________________________________________________________________________________
concatenate_12 (Concatenate)    (None, 16)           0           bidirectional_12[0][0]           
                                                                 bidirectional_12[1][0]           
__________________________________________________________________________________________________
dense_73 (Dense)                (None, 4)            68          concatenate_12[0][0]             
_________